In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/My\ Drive/School/CISC_452/Project

data  glove  LSTM.py  preprocessing  README.md	requirements.txt  saved_models


In [0]:
pip install -r /content/gdrive/My\ Drive/School/CISC_452/Project/requirements.txt


     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 2.4MB 25.9MB/s 
     |████████████████████████████████| 2.9MB 61.7MB/s 
     |████████████████████████████████| 1.5MB 40.1MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 235kB 67.4MB/s 
     |████████████████████████████████| 512kB 64.6MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 3.8MB 57.3MB/s 
     |████████████████████████████████| 86.3MB 133kB/s 
     |████████████████████████████████| 450kB 63.0MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449909 sha256=d167436b716d741ce883bb6d59a77fbc1dffdc9224556dca591829db977584f2
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
ERROR: google-colab 1.0.0 has requirement google-auth~

In [4]:
pip install tensorflow-gpu==2.0.0rc0

     |████████████████████████████████| 380.5MB 43kB/s 
     |████████████████████████████████| 4.3MB 45.1MB/s 
     |████████████████████████████████| 501kB 46.1MB/s 


In [2]:
import sys
import os
sys.path.append('/content/gdrive/My Drive/School/CISC_452/Project')
os.chdir('/content/gdrive/My Drive/School/CISC_452/Project')
import numpy                    as np
import preprocessing.preprocessing_helpers as preproc
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Flatten, Embedding, SpatialDropout1D, GlobalMaxPool1D, Bidirectional, Dropout
from tensorflow.keras.models import load_model
import tensorflow as tf
import pandas as pd

# VOCAB_SIZE = 5000

class Toxic_Comment_LSTM(object):
    def __init__(self,x_train=None,y_train=None,x_test=None,y_test=None,embedding_matrix=None,max_length=None,saved_model=None):
        super().__init__()
        self.max_length     = max_length
        self.model          = self.define_model(embedding_matrix,saved_model=saved_model)
        self.x_train        = x_train
        self.y_train        = y_train
        self.x_test         = x_test
        self.y_test         = y_test
    # #  REGULAR JMODEL
    # def define_model(self,embedding_matrix,saved_model=None):
    #     if saved_model is None:
    #         print("--- Initializing Model ---")
    #         VOCAB_SIZE = embedding_matrix.shape[0]
    #         model = Sequential()
    #         embedding_layer = Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
    #         model.add(embedding_layer)
    #         model.add(SpatialDropout1D(0.2))
    #         model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
    #         model.add(Dense(6, activation='sigmoid'))
    #         model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #         return model
    #     print("--- Loading Model from {} ---".format(saved_model))
    #     model = preproc.load_h5_model(saved_model)
    #     if model is None: # If the filepath is wrong or the model hasn't actually been defined earlier
    #         print("--- no model found, initializing from scrach ---")
    #         return self.define_model(embedding_matrix,saved_model=None)
    #     return model

    # Multi LSTM
    def define_model(self,embedding_matrix,saved_model=None):
        if saved_model is None:
            print("--- Initializing Model ---")
            VOCAB_SIZE = embedding_matrix.shape[0]
            model = Sequential()
            embedding_layer = Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
            model.add(embedding_layer)
            model.add(SpatialDropout1D(0.2))
            model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
            model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
            model.add(Dense(6, activation='sigmoid'))
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            return model
        print("--- Loading Model from {} ---".format(saved_model))
        model = preproc.load_h5_model(saved_model)
        if model is None: # If the filepath is wrong or the model hasn't actually been defined earlier
            print("--- no model found, initializing from scrach ---")
            return self.define_model(embedding_matrix,saved_model=None)
        return model
  
    def train(self,x_train=None,y_train=None):
        x_train = self.x_train if x_train is None else x_train
        y_train = self.y_train if y_train is None else y_train
        epochs = 2
        batch_size = 64
        # callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001,verbose=1)
        history = self.model.fit(x_train,y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,verbose=1)

    def validate(self):
        # Evaluate the model on the test data using `evaluate`
        print('\n# Evaluate on test data')
        print(self.x_test.shape,self.y_test.shape)
        results = self.model.evaluate(self.x_test, self.y_test, batch_size=128,verbose=0)
        print('test loss, test acc:', results)

        # Generate predictions (probabilities -- the output of the last layer)
        # on new data using `predict`
        shuffle_idx = np.random.choice(np.arange(self.x_test.shape[0]), 10, replace=False)
        x_sample, y_sample = self.x_test[shuffle_idx],self.y_test[shuffle_idx]
        print('\n# Generate predictions for {} samples'.format(x_sample.shape[0]))
        predictions = self.model.predict(x_sample)
        for i,(prediction,target) in enumerate(zip(predictions,y_sample)):
            chosen = np.argmax(prediction)
            actual = np.argmax(target)
            print("\t* prediction {} -> {}, actual -> {}".format(i,chosen,actual))
            print("\t\t* {}".format(actual))
        print('predictions shape:', predictions.shape)

    def save_model(self,file_path="saved_models/toxic_comment_LSTM.h5"):
        
        preproc.save_h5_model(file_path,self.model)
        print("--- model saved to {} ---".format(file_path))

if __name__ == "__main__":
    labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # print('--- Reading test data ---')
    # test_df, test_max_length = preproc.return_data('./data/cleaned_test_labels.csv')
    # test_data = np.array(test_df['cleaned_text'])
    # test_labels = np.array(test_df[labels])

    text_df, max_length   = preproc.return_data('./data/{}.csv'.format("upsampled_downsampled_train"))
    train  = np.array(text_df['cleaned_text'])
    t = Tokenizer(filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~')
    t.fit_on_texts(train)
    train = t.texts_to_sequences(train)
    train = pad_sequences(train, maxlen=max_length, padding='post')


    # # tokenize the test data
    # test_data = t.texts_to_sequences(test_data)
    # test_data = pad_sequences(test_data, maxlen=test_max_length, padding='post')



    embedding_matrix = np.load('./data/embedding_matrix.npy')

    mask = np.random.rand(len(train)) < 0.8

    x_train, x_test = train[mask], train[~mask] # split data into train test splits
    y_train, y_test = np.array(text_df[mask][labels]), np.array(text_df[~mask][labels])
    toxic_Comment_LSTM = Toxic_Comment_LSTM(x_test=x_test,y_test=y_test, embedding_matrix=embedding_matrix, saved_model='saved_models/toxic_comment_LSTM_multiLSTM.h5')

    # print('--- Predicting ---')
    # toxic_Comment_LSTM.model.evaluate(test_data, test_labels)
    print("--- Training Model ---")
    toxic_Comment_LSTM.model.fit(x_train, y_train)
    print("--- Saving Model ---")
    toxic_Comment_LSTM.save_model('saved_models/toxic_comment_LSTM_multiLSTM.h5')




FileNotFoundError: ignored